# Add Features

In [1]:
import psycopg2 as pg
import pandas as pd
import numpy as np

from psycopg2.extras import execute_values

1. Historical Tournament Win
2. Score Margin (Actual + %)

In [2]:
# Just looking to see what's in there already

conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

# pull regular season and tourney data to build indexes
query = '''
    SELECT *
    FROM prod.features
    LIMIT 20
'''

df = pd.read_sql_query(query, conn)
conn.close()

In [3]:
df.head()

,Season,DayNum,Team,Opponent,Outcome,Score,OpponentScore,NumOT,WLoc,Season Type,...,WinPct,OpponentWinPct,WinPctDiff,AvgPointsFor,AvgPointsAgainst,AvgNetPointsFor,SeedDiff,OpponentAvgPointsFor,OpponentAvgPointsAgainst,OpponentAvgNetPointsFor
0,2009,136,1437,1110,1.0,80.0,67.0,0.0,N,NCAA Tourney,...,0.787879,0.774194,0.013685,76.5455,67.0000,9.54545,-11,64.4516,58.4194,6.03226
1,2018,999,1112,1328,NaN,NaN,NaN,NaN,None,NCAA Tourney,...,0.794118,0.580645,0.213472,80.8824,71.1765,9.70588,-6,85.1613,81.6452,3.51613
2,2018,999,1112,1345,NaN,NaN,NaN,NaN,None,NCAA Tourney,...,0.794118,0.823529,-0.029412,80.8824,71.1765,9.70588,2,81.1471,65.6471,15.50000
3,2018,999,1112,1347,NaN,NaN,NaN,NaN,None,NCAA Tourney,...,0.794118,0.625000,0.169118,80.8824,71.1765,9.70588,-12,66.9375,64.8750,2.06250
4,2018,999,1112,1371,NaN,NaN,NaN,NaN,None,NCAA Tourney,...,0.794118,0.656250,0.137868,80.8824,71.1765,9.70588,-4,79.0000,73.3125,5.68750


In [4]:
df.columns

Index([u'Season', u'DayNum', u'Team', u'Opponent', u'Outcome', u'Score',
       u'OpponentScore', u'NumOT', u'WLoc', u'Season Type', u'holdout_s1',
       u'holdout_s2', u'AvgRank', u'OpponentAvgRank', u'TwoPointPct_Team',
       u'ThreePointPct_Team', u'FreeThrowPct_Team', u'OffensiveRebounds_Team',
       u'DefensiveRebounds_Team', u'TwoPointPct_Opponent',
       u'ThreePointPct_Opponent', u'FreeThrowPct_Opponent',
       u'OffensiveRebounds_Opponent', u'DefensiveRebounds_Opponent', u'WinPct',
       u'OpponentWinPct', u'WinPctDiff', u'AvgPointsFor', u'AvgPointsAgainst',
       u'AvgNetPointsFor', u'SeedDiff', u'OpponentAvgPointsFor',
       u'OpponentAvgPointsAgainst', u'OpponentAvgNetPointsFor'],
      dtype='object')

## Historical Tournament Wins

Don't use the tournament data from the database any more since it's capped at 2003

In [51]:
conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

# pull tournament wins by team and season
#query = '''
#    SELECT "Team", "Season", SUM("Outcome")
#    FROM prod.features
#    WHERE "Season Type" = 'NCAA Tourney'
#    GROUP BY "Team", "Season"
#'''

query = '''
    SELECT "WTeamID","Season", COUNT(*)
    FROM prod."ConferenceTourneyGames"
    GROUP BY "Season","WTeamID"
'''

tour_wins_df = pd.read_sql_query(query, conn)
conn.close()

In [52]:
tour_wins_df.columns = ['TeamID','Season','TourWins']
tour_wins_df.head()

,TeamID,Season,TourWins
0,1190,2004,3
1,1104,2011,1
2,1358,2003,2
3,1132,2015,1
4,1103,2016,2


In [53]:
tour_wins_df.shape

(2829, 3)

In [55]:
for t in tour_wins_df.TeamID.unique():
    tour_wins_df.loc[tour_wins_df.shape[0]] = [t,2018,0]

In [56]:
tour_wins_df.tail()

,TeamID,Season,TourWins
3164,1343,2018,0
3165,1244,2018,0
3166,1183,2018,0
3167,1297,2018,0
3168,1410,2018,0


In [79]:
tour_wins_df.Season.unique()

array([2004, 2011, 2003, 2015, 2016, 2002, 2017, 2006, 2012, 2008, 2009,
       2005, 2010, 2013, 2014, 2001, 2007, 2018])

Pulling data from the CSV instead

In [169]:
#using the csv instead since the DB didn't have data before 2003
df_tour = pd.read_csv('/Users/elyliu/Documents/mids/w207/Spring2018-Section3/Projects/Final Kaggle Proj 2/mens-machine-learning-competition-2018/Stage2UnchangedDataFiles/NCAATourneyCompactResults.csv')

In [170]:
df_tour.Season.unique()

array([1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006,
       2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017])

In [171]:
df_tour.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [172]:
df_tour_wins = df_tour.groupby(['WTeamID', 'Season']).count()
df_tour_wins.drop(labels=['WScore','LTeamID','LTeamID','LScore','WLoc','NumOT'], inplace=True, axis=1)
df_tour_wins.reset_index(inplace=True)
df_tour_wins.columns = ['TeamID','Season','TourWins']
df_tour_wins.head()

,TeamID,Season,TourWins
0,1104,1985,2
1,1104,1986,2
2,1104,1987,2
3,1104,1990,2
4,1104,1991,2


Filling in the 0 win years

In [173]:
for t in df_tour_wins.TeamID.unique():
    for y in range(1985,2019):
        if len(df_tour_wins[(df_tour_wins.TeamID==t)&(df_tour_wins.Season==y)]) < 1:
            df_tour_wins.loc[df_tour_wins.shape[0]] = [t,y,0]

Pull out the features database so I can attach the tour win data

In [177]:
conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

# pull tournament wins by team and season
query = '''
    SELECT "Team", "Opponent", "Season"
    FROM prod.features
'''

df = pd.read_sql_query(query, conn)
conn.close()

In [178]:
df.head()

,Team,Opponent,Season
0,1104,1328,2003
1,1272,1393,2003
2,1328,1104,2003
3,1393,1272,2003
4,1186,1458,2003


In [179]:
df.shape

(170600, 3)

Adding a cume tour win column

In [180]:
def cumulativeTourWins(row):
    return df_tour_wins[(df_tour_wins.TeamID==row.TeamID)&(df_tour_wins.Season<row.Season)].TourWins.sum()

df_tour_wins['cumeTourWins'] = df_tour_wins.apply(cumulativeTourWins, axis=1)
df_tour_wins.head()

,TeamID,Season,TourWins,cumeTourWins
0,1104,1985,2,0
1,1104,1986,2,2
2,1104,1987,2,4
3,1104,1990,2,6
4,1104,1991,2,8


Pivoting the table for faster lookup

In [182]:
cume_tour_wins_df_pivot = df_tour_wins.pivot(index='TeamID', columns='Season', values='cumeTourWins')

# this line may not be needed anymore since I filled in all the empty years
cume_tour_wins_df_pivot = cume_tour_wins_df_pivot.transpose().fillna(method='ffill').fillna(0).transpose()

cume_tour_wins_df_pivot.head()

Season,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
TeamID,,,,,,,,,,,,,,,,,,,,,
1104,0,2,4,6,6,6,8,10,11,11,...,18,18,18,18,18,18,18,18,18,18
1107,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
1112,0,0,0,0,4,6,7,9,9,9,...,39,41,41,44,44,46,49,52,52,54
1113,0,0,0,0,0,0,0,1,1,1,...,4,5,5,5,5,5,5,5,5,5
1114,0,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2,2


In [206]:
# just code for testing
df_tour_wins[df_tour_wins.TeamID==1166].sort_values(by=['Season'])

,TeamID,Season,TourWins,cumeTourWins
1950,1166,1985,0,0
1951,1166,1986,0,0
1952,1166,1987,0,0
1953,1166,1988,0,0
1954,1166,1989,0,0
1955,1166,1990,0,0
157,1166,1991,1,0
1956,1166,1992,0,1
1957,1166,1993,0,1
1958,1166,1994,0,1


In [188]:
# for teams with 0 tour wins
for t in df.Team.unique():
    if len(cume_tour_wins_df_pivot[cume_tour_wins_df_pivot.index==t]) < 1:
        cume_tour_wins_df_pivot.loc[t] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [189]:
cume_tour_wins_df_pivot.tail()

Season,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
TeamID,,,,,,,,,,,,,,,,,,,,,
1297,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1213,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [190]:
teamWins = cume_tour_wins_df_pivot.lookup(df.Team,df.Season)
oppWins = cume_tour_wins_df_pivot.lookup(df.Opponent,df.Season)
tourWinDiff = teamWins - oppWins
new_info_df = pd.DataFrame([teamWins,oppWins,tourWinDiff]).transpose()
new_info_df.head()

,0,1,2
0,14,21,-7
1,11,27,-16
2,21,14,7
3,27,11,16
4,0,6,-6


In [69]:
# old code too slow
#def tourWinsBefore(row):
#    teamWins = tour_wins_df[(tour_wins_df.TeamID==row.Team)&(tour_wins_df.Season==row.Season)].cumeTourWins
#    oppWins = tour_wins_df[(tour_wins_df.TeamID==row.Opponent)&(tour_wins_df.Season==row.Season)].cumeTourWins
#    return teamWins, oppWins, teamWins - oppWins

#df_tour_wins_temp = pd.DataFrame()
#df_tour_wins_temp = pd.DataFrame(df.apply(tourWinsBefore, axis=1))[0].apply(pd.Series)
#df_tour_wins_temp.columns = ['TeamTWins','OppTWins','TourWinDiff']
#df_tour_wins_temp.tail()

In [191]:
df = pd.concat([df, new_info_df], axis=1)
df.head()

,Team,Opponent,Season,0,1,2
0,1104,1328,2003,14,21,-7
1,1272,1393,2003,11,27,-16
2,1328,1104,2003,21,14,7
3,1393,1272,2003,27,11,16
4,1186,1458,2003,0,6,-6


In [192]:
df.tail()

,Team,Opponent,Season,0,1,2
170595,1462,1438,2018,27,18,9
170596,1462,1439,2018,27,2,25
170597,1462,1452,2018,27,19,8
170598,1462,1455,2018,27,12,15
170599,1462,1460,2018,27,0,27


In [194]:
df.columns = ['Team','Opponent','Season','TeamTWins','OppTWins','TourWinDiff']

In [195]:
df.head()

,Team,Opponent,Season,TeamTWins,OppTWins,TourWinDiff
0,1104,1328,2003,14,21,-7
1,1272,1393,2003,11,27,-16
2,1328,1104,2003,21,14,7
3,1393,1272,2003,27,11,16
4,1186,1458,2003,0,6,-6


In [197]:
datarows = []
for i in df.itertuples(index=False):
    row = (int(i.Season), int(i.Team), int(i.Opponent), int(i.TeamTWins), int(i.OppTWins), int(i.TourWinDiff))
    datarows.append(row)

In [198]:
alter = '''
    ALTER TABLE prod.features
      ADD COLUMN IF NOT EXISTS "TourWins" REAL,
      ADD COLUMN IF NOT EXISTS "OpponentTourWins" REAL,
      ADD COLUMN IF NOT EXISTS "TourWinsDiff" REAL
'''

update = '''
    UPDATE prod.features as f
       SET "TourWins" = data."TeamTWins",
           "OpponentTourWins" = data."OppTWins",
           "TourWinsDiff" = data."TourWinDiff"
      FROM (VALUES %s) AS data (
             "Season",
             "Team",
             "Opponent",
             "TeamTWins",
             "OppTWins",
             "TourWinDiff"
             )
     WHERE f."Season" = data."Season"
       and f."Team" = data."Team"
       and f."Opponent" = data."Opponent"
'''

conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

c = conn.cursor()
c.execute(alter)
execute_values(c, update, datarows, page_size=8000)
conn.commit()
conn.close()